<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

# __*Labelbox Connector for Databricks Tutorial Notebook*__

# _**Creating Data Rows with Metadata, Attachments and Annotations with LabelSpark**_

<td>
<a href="https://github.com/Labelbox/labelspark/blob/master/notebooks/full-demo.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

Thanks for trying out the Databricks and Labelbox Connector! You or someone from your organization signed up for a Labelbox trial through Databricks Partner Connect. This notebook was loaded into your Shared directory to help illustrate how Labelbox and Databricks can be used together to power unstructured data workflows. 

Labelbox can be used to rapidly annotate a variety of unstructured data from your Data Lake ([images](https://labelbox.com/product/image), [video](https://labelbox.com/product/video), [text](https://labelbox.com/product/text), and [geospatial tiled imagery](https://docs.labelbox.com/docs/tiled-imagery-editor)) and the Labelbox Connector for Databricks makes it easy to bring the annotations back into your Lakehouse environment for AI/ML and analytical workflows. 

If you would like to watch a video of the workflow, check out our [Data & AI Summit Demo](https://databricks.com/session_na21/productionizing-unstructured-data-for-ai-and-analytics). 


<img src="https://labelbox.com/static/images/partnerships/collab-chart.svg" alt="example-workflow" width="800"/>

<h5>Questions or comments? Reach out to us at [ecosystem+databricks@labelbox.com](mailto:ecosystem+databricks@labelbox.com)

## _**Documentation**_

### **Data Rows**
_____________________

**Requirements:**

- A `row_data` column - This column must be URLs that point to the asset to-be-uploaded

- Either a `dataset_id` column or an input argument for `dataset_id`
  - If uploading to multiple datasets, provide a `dataset_id` column 
  - If uploading to one dataset, provide a `dataset_id` input argument
    - _This can still be a column if it's already in your CSV file_

**Recommended:**
- A `global_key` column
  - This column contains unique identifiers for your data rows
  - If none is provided, will default to your `row_data` column
- An `external_id` column
  - This column contains non-unique identifiers for your data rows
  - If none is provided, will default to your `global_key` column  

**Optional:**
- A `project_id` columm or an input argument for `project_id`
  - If batching to multiple projects, provide a `project_id` column
  - If batching to one project, provide a `project_id` input argument
    - _This can still be a column if it's already in your CSV file_

### **Attachments**
_____________________

For attachments, the column name must be " `attachment` + `divider` + `attachment_type` + `divider` + `column_name` "
  - Example: `attachment///raw_text///sample_column_name`
  - `attachment_type` must be one of the following:
    - `image`, `video`, `raw_text`, `html`, `text_url`


Values for attachments must correspond with the attachment type per Labelbox docs
  - More here: 
    - [Labelbox docs on attachments](https://docs.labelbox.com/docs/asset-attachments)

### **Metadata**
_____________________
For metadata, the column name must be " `metadata` + `divider` + `metadata_type` + `divider` + `metadata_field_name` "
  - Example: `metadata///string///sample_metadata_field_name`
  - `metadata_type` must be one of the following:
    - `string`, `enum`, `datetime`, `number` 
  - If the `metadata_field_name` doesn't exist yet in Labelbox, LabelSpark will create it for you


The values for metadata fields must correspond with the metadata type per Labelbox docs
  - More here:
    - [Labelbox definition of metadata](https://docs.labelbox.com/docs/datarow-metadata)
    - [Labelbox docs on creating metadata](https://docs.labelbox.com/docs/createmodify-metadata-schema)

### **Annotations**
_____________________

*Note:*
*There must also be a `project_id` column, or an input argument for `project_id` when using LabelSpark to upload annotations*

*There must also be an `upload_method` provided when using LabelSpark*
  - *`upload_method` must be one of the following:*
    - *`"mal"` (uploads annotations as pre-labels)*
    - *`"import"` (uploads annotations as submitted labels)*
    

- For annotations, the column name must be `annotation` + `divider` + `annotation_type` + `divider` + `top_level_feature_name`
  - Example: `annotation///bbox///bbox_tool_name` where, in this case, the bounding tool name in your Labelbox ontology is "bbox_tool_name"
  - `annotation_type` must be one of the following:
    - `bbox`, `polygon`, `point`, `mask`, `line`, `named-entity`, `radio`, `checklist`, `text`
- Values for annotations must correspond with the following, per annotation type:

_____________________
_____________________

**Row-Level Formats for Tool Annotations**
- `bbox` (this example is two annotations)
```
[
        [[top, left, height, width], [nested_classification_name_paths]], 
        [[top, left, height, width], [nested_classification_name_paths]]
]
```
- `polygon` (this example is two annotations)
```
[
        [[(x, y), (x, y),...(x, y)], [nested_classification_name_paths]], 
        [[(x, y), (x, y),...(x, y)], [nested_classification_name_paths]]
]
```
- `line` (this example is two annotations)
```
[
        [[(x, y), (x, y),...(x, y)], [nested_classification_name_paths]], 
        [[(x, y), (x, y),...(x, y)], [nested_classification_name_paths]]
]
```
- `point` (this example is two annotations)
```
[
        [[x, y], [nested_classification_name_paths]], 
        [[x, y], [nested_classification_name_paths]]
]
```
- `mask` (this example is two annotations)
```
[
        [[URL, colorRGB], [nested_classification_name_paths]], 
        [[URL, colorRGB], [nested_classification_name_paths]]
]
                      OR
[
        [[numpy_array, colorRGB], [nested_classification_name_paths]], 
        [[numpy_array, colorRGB], [nested_classification_name_paths]]
]
                      OR
[
        [[png_bytes, None], [nested_classification_name_paths]], 
        [[png_bytes, None], [nested_classification_name_paths]]
]
```
- `named-entity` (this example is two annotations)
```
[
        [[start, end], [nested_classification_name_paths]], 
        [[start, end], [nested_classification_name_paths]]
]
```

**Row-Level Formats for Classification Annotations**
- `radio`, `checklist` and `text`
```
[[answer_name_paths]]
```
  - Note: the last string in a text name path is the text answer value itself
_____________________
_____________________

## _**Code**_

Install LabelSpark

In [0]:
%pip install labelspark -q

In [0]:
import labelspark as ls
# Imported to create an example ontology - not required for typical runs of LabelSpark
from labelbox.schema.ontology import OntologyBuilder, Tool, Classification, Option

In [0]:
table_name = "labelspark_full_demo" # Name to register test table under
csv_path = "https://raw.githubusercontent.com/Labelbox/labelspark/master/datasets/full-import.csv" # Path to your CSV file
api_key = ""

Load a CSV as a Spark Table

In [0]:
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('labelspark-demo').getOrCreate() # Create a Spark Session
df = pd.read_csv(csv_path) # Read CSV in as Pandas DataFrame
table = spark.createDataFrame(df) # Convert your Pandas DataFrame into a Spark DataFrame

Register your Spark Table (for this demo, we will only save as a temporary table)

In [0]:
tblList = spark.catalog.listTables()

if not any([x.name == table_name for x in tblList]):
  table.createOrReplaceTempView(table_name)
  print(f"Registered table: {table_name}")

In [0]:
display(sqlContext.sql(f"select * from {table_name} LIMIT 5"))

Create a project, dataset and ontology

In [0]:
client = ls.Client(lb_api_key=api_key)

In [0]:
project = client.lb_client.create_project(name="LabelSpark-full-demo")
dataset = client.lb_client.create_dataset(name="LabelSpark-full-demo")

In [0]:
ontology_builder = OntologyBuilder(
    classifications=[ 
        Classification( # Radio classification
            class_type=Classification.Type.RADIO, instructions="sample_radio_question", 
            options=[Option(value="sample_radio_answer_1"), Option(value="sample_radio_answer_2")]
        ),
        Classification( # Checklist classification
            class_type=Classification.Type.CHECKLIST, instructions="sample_checklist_question", 
            options=[Option(value="sample_checklist_answer_1"), Option(value="sample_checklist_answer_2")]
        ), 
        Classification( # Text classification
            class_type=Classification.Type.TEXT, instructions="sample_free_text_question"
        ),
        Classification( # Radio classification where one answer has a nested radio classification
            class_type=Classification.Type.RADIO, instructions="sample_nested_radio_question",
            options=[
                Option(
                    value="sample_branch_radio_answer_1", 
                    options=[
                        Classification(
                            class_type=Classification.Type.RADIO, instructions="sample_sub_radio_question", 
                            options=[Option("sample_sub_radio_answer_1"), Option("sample_sub_radio_answer_2")]
                        )
                    ]
                ), 
                Option(value="sample_leaf_radio_answer_2")
            ]
        )
    ],
    tools=[ # List of Tool objects
        Tool( # Bounding Box tool
            tool=Tool.Type.BBOX, name="sample_bounding_box"), 
        Tool( # Bounding Box tool with a nested text classification
            tool=Tool.Type.BBOX,  name="sample_nested_bounding_box",
            classifications=[
                Classification(class_type=Classification.Type.TEXT, instructions="sample_tool_sub_text_question"),]
        ),
        Tool( # Polygon tool
            tool=Tool.Type.POLYGON, name="sample_polygon"
        ),
        Tool( # Polygon tool with a nested radio classification
            tool=Tool.Type.POLYGON, name="sample_nested_polygon",
            classifications=[
                Classification(
                    class_type=Classification.Type.RADIO, instructions="sample_tool_sub_radio_question",
                    options=[Option("sample_sub_radio_answer_1"), Option("sample_sub_radio_answer_2")]
                ),
            ]            
        ),        
        Tool( # Segmentation mask tool given the name "mask"
            tool=Tool.Type.SEGMENTATION, name="sample_segmentation_mask"
        ),
 	      Tool( # Point tool given the name "point"
            tool=Tool.Type.POINT, name="sample_point"
        ), 
        Tool( # Polyline tool given the name "line"
            tool=Tool.Type.LINE, name="sample_polyline"
        )
    ]
)

ontology = client.lb_client.create_ontology("LabelSpark-full-demo", ontology_builder.asdict())

project.setup_editor(ontology)

Upload to Labelbox

In [0]:
results = client.create_data_rows_from_table(
    table = table,
    dataset_id = dataset.uid,
    project_id = project.uid,
    upload_method = "import", # Must be either "import" or "mal"
    skip_duplicates = False, # If True, will skip data rows where a global key is already in use
    mask_method = "png", # Input masks must be either "png", "url", or "array"
    verbose = True, # If True, prints information about code execution
)